# **Шаг 4:** Нейросети

## **4.1** Тренировка предобученных моделей на RSNA датасете

Импортируем библиотеки:

In [3]:
import pandas as pd
import numpy as np

In [4]:
from PIL import Image
from tqdm.notebook import tqdm
## VV: библиотеки для анализа изображений и построения нейронной сети и подгрузки данных
import random
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader

try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    #!pip install -q torchinfo
    !pip install /kaggle/input/rsna-packages/torchinfo-1.7.2-py3-none-any.whl
    from torchinfo import summary
    
    
from torch.optim.lr_scheduler import ReduceLROnPlateau
from albumentations import HorizontalFlip, VerticalFlip
import wandb

In [5]:
def seed_everything(seed):
    random.seed(seed) # фиксируем генератор случайных чисел
    os.environ['PYTHONHASHSEED'] = str(seed) # фиксируем заполнения хешей
    np.random.seed(seed) # фиксируем генератор случайных чисел numpy
    torch.manual_seed(seed) # фиксируем генератор случайных чисел pytorch
    torch.cuda.manual_seed(seed) # фиксируем генератор случайных чисел для GPU
    torch.backends.cudnn.deterministic = True # выбираем только детерминированные алгоритмы (для сверток)
    torch.backends.cudnn.benchmark = False # фиксируем алгоритм вычисления сверток

seed_everything(12345)

Загрузим предобработанные изображения и метаданные.

In [6]:
df_train_cut = pd.read_csv('/kaggle/input/rsna-breast-cancer-pt-2-image-processing/RSNA-cropped-png-train/df_train.csv')
# replace '/kaggle/working' with '/kaggle/input/rsna-breast-cancer-pt-2-image-processing/'


Зафиксируем генератор случайных чисел для воспроизводимости результатов:

Создадим конфигурацию эксперимента:

In [7]:
class CFG:


  api = "0936d7194d5bdad4c06a0dc37bd3d4360becade1"
  project = "Kaggle_RSNA_challenge_final"
  entity = "veronikavoronova"
  num_epochs = 18 # 50
  train_batch_size = 40
  test_batch_size = 40
  num_workers = 4
  lr = 3e-4
  classes = ('healthy','cancer')
  wandb = False # True
    
def class2dict(f):
  return dict((name, getattr(f, name)) for name in dir(f) if not name.startswith('__'))

Создадим класс для работы с датасетом. Наследуемся от класса Dataset https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset

In [8]:
## VV: адаптировано из https://www.kaggle.com/code/andradaolteanu/rsna-breast-cancer-eda-pytorch-baseline
class RSNADataset(Dataset):
    
    def __init__(self, dataframe, path_ds, csv_columns, augment = False, is_train=True):
        self.dataframe = dataframe
        self.is_train = is_train
        self.path_ds = path_ds
        self.csv_columns = csv_columns
        if augment:
            self.transform = transforms.Compose([#transforms.RandomHorizontalFlip(p = 0.5),
                                                 #transforms.RandomVerticalFlip(p = 0.3),
                                                 transforms.RandomPerspective(p=0.1),
                                                 transforms.RandomRotation(degrees=(-90,90)),
                                                 transforms.Grayscale(num_output_channels=3),
                                                 transforms.ToTensor()])
        else:
            self.transform = transforms.Compose([transforms.Grayscale(num_output_channels=3),
                                                 transforms.ToTensor()])
        
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, index):
        self.dataframe['new_image_path'] =  self.path_ds + \
        self.dataframe.patient_id.map(str) + '_' + \
        self.dataframe.image_id.map(str) + '.png'

        image_path = self.dataframe['new_image_path'].iloc[index]
        im_frame = Image.open(image_path)
        transf_image = (self.transform(im_frame))        
        csv_data = (np.array(self.dataframe.iloc[index][self.csv_columns].values, 
                            dtype=np.float32)) #torch.FloatTensor
        if not self.is_train:
            self.dataframe = self.dataframe.replace({'R': 0, 'L' : 1})
            subm_data = (np.array(self.dataframe.iloc[index][['patient_id','laterality']].values,dtype=np.float32)) #,'laterality'
        
        if self.is_train:
            label = self.dataframe['cancer'].iloc[index]
            return transf_image, csv_data, label

        else:
            return transf_image, csv_data, subm_data


Создадим экземпляр класса и проверим, что все работает

In [9]:
class clr:
    S = '\033[1m' + '\033[91m'
    E = '\033[0m'
    
rsna_train = RSNADataset(df_train_cut.iloc[0:10],
                         augment = False,
                         path_ds = '/kaggle/input/rsna-breast-cancer-pt-2-image-processing/RSNA-cropped-png-train_gabor/',
                         csv_columns = ['age','implant'])

rsna_dataloader = DataLoader(rsna_train, 
                             batch_size=CFG.train_batch_size, 
                             shuffle=False)
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

Конструктор класса работает корректно. Теперь попробуем разбить нашу выборку на две части (тренировочную и тестовую) в соотношении 70 на 30.

In [10]:
PATH_DS = '/kaggle/input/rsna-breast-cancer-pt-2-image-processing/RSNA-cropped-png-train_gabor/'

def init_ds(ds, path_ds, csv_columns = ['age','implant'], augment=False):
     rsna_train = RSNADataset(ds,
                              path_ds = PATH_DS,
                              csv_columns = ['age','implant'])
     if augment:
        rsna_train = RSNADataset(ds,
                              path_ds = PATH_DS,
                              augment = True,
                              csv_columns = ['age','implant'])   

     train_ds, valid_ds = torch.utils.data.random_split(dataset = rsna_train, 
                                                       lengths = [3793-1000,1000], 
                                                       generator = 
                                                       torch.Generator().manual_seed(42))
     train_loader = DataLoader(train_ds, 
                              batch_size = CFG.train_batch_size, 
                              shuffle = False, 
                              drop_last = True)

     valid_loader = DataLoader(valid_ds, 
                              batch_size = CFG.train_batch_size,
                              shuffle = False, 
                              drop_last = True)
     return train_loader, valid_loader

In [11]:
train_loader, valid_loader = init_ds(df_train_cut, path_ds = PATH_DS)

Теперь попробуем несколько готовых решений с fine tuning

In [12]:
## VV: to modify https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html
## about pretrained model usage for dicom https://www.researchgate.net/post/Can-we-use-pre-trained-models-like-InceptionV3-VGG16-on-medical-image-datasets
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet50
        """
        model_ft = models.resnet50(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "vgg11":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        model_ft.num_classes = num_classes
        input_size = 224
    elif model_name == "vgg16":
        """ VGG16_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        model_ft.num_classes = num_classes
        input_size = 224
    elif model_name == "vgg19":
        """ VGG19_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "efficientnet":
        """ efficientnet_b5
        """
        model_ft = models.efficientnet_b5(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[1].in_features
        model_ft.classifier[1] = nn.Linear(num_ftrs, num_classes)
        model_ft.num_classes = num_classes
        input_size = 224

    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size

Загрузка моделей работает адекватно.
Теперь создадим класс модели, который на вход принимает одну из предобученных моделей и дополняет выход сети предсказаниями по метаданным.

In [13]:
class RSNA_model(nn.Module):
    def __init__(self,model,csv_use = False, n_neu=4,  csv_neu=2):
        super().__init__()
        self.n_neu = n_neu       
        self.csv_neu = csv_neu
        self.csv_use = csv_use
        self.features = model
        self.num_classes = model.num_classes
        self.csv = nn.Sequential(nn.Linear(self.csv_neu, self.n_neu),
                                 nn.ReLU(),
                                 nn.Linear(self.n_neu, 100),
                                 nn.ReLU())
        if self.csv_use:
            self.classif = nn.Linear(model.num_classes + 100, 2)
        else:
            self.classif = nn.Linear(model.num_classes, 2)
        
        
    def forward(self, img, meta):
        img_1 = self.features(img)
        meta_1 = self.csv(meta)
        if self.csv_use:
            x = torch.cat((img_1, meta_1), dim=1)       
        else:
            x = img_1
        x   = self.classif(x)
        return x

Определим функцию обучения.

In [14]:
## VV: частично использован код из наших семинаров
def train(model, train_loader):
        
    model = model.train() 
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr = CFG.lr)
    scheduler = ReduceLROnPlateau(optimizer=optimizer, mode='max', 
                                      patience=1, verbose=True, factor=0.4)

    train_accuracy,train_loss = 0, 0
    n_batch_train, len_batch_train = len(train_loader), len(train_loader.dataset)
    
    for batch_idx, (data, meta, target) in tqdm(enumerate(train_loader), total=n_batch_train):
        train_accuracy_batch = 0
        data, meta, target = data.to(device), meta.to(device), target.to(device)
        optimizer.zero_grad()
        out = model(data, meta)
        pred = out.argmax(dim=1, keepdim=True)
        train_loss_batch = criterion(out, target) ## VV: for BCElogitloss https://discuss.pytorch.org/t/how-to-cast-a-tensor-to-another-type/2713/2
        train_accuracy_batch += pred.eq(target.view_as(pred)).sum().item()
        train_loss_batch.backward()
            
        train_accuracy += train_accuracy_batch    
        train_loss += train_loss_batch
            
        optimizer.step()
        
    tqdm.write('Train set: Average loss: {:.4f}, Accuracy: {:.2f}%'.format(
        train_loss/n_batch_train, 100. * train_accuracy/len_batch_train))
    
    if CFG.wandb:
        wandb.log({'train_loss': train_loss/n_batch_train,
                   'train_accuracy': train_accuracy/len_batch_train})
    return train_accuracy, train_loss

In [15]:
def valid(model, valid_loader):
    model = model.eval() 
    valid_accuracy,valid_loss = 0, 0
    n_batch_valid, len_batch_valid = len(valid_loader), len(valid_loader.dataset)
    criterion = torch.nn.CrossEntropyLoss() 
    
    with torch.no_grad():
        for batch_idx, (data, meta, target) in tqdm(enumerate(valid_loader), total=n_batch_valid):
            valid_accuracy_batch = 0
            data, meta, target = data.to(device), meta.to(device), target.to(device)
            model = model.eval()
            out = model(data, meta)
            pred = out.argmax(dim=1, keepdim=True)
            valid_loss_batch = criterion(out, target)
            valid_accuracy_batch += pred.eq(target.view_as(pred)).sum().item()
            
            valid_accuracy += valid_accuracy_batch
            valid_loss += valid_loss_batch
            
    tqdm.write('Valid set: Average loss: {:.4f}, Accuracy: {:.2f}%'.format(
        valid_loss/n_batch_valid, 100. * valid_accuracy/len_batch_valid))
    
    if CFG.wandb:
        wandb.log({'valid_loss': valid_loss/n_batch_valid,
                   'valid_accuracy': valid_accuracy/len_batch_valid})
    return valid_accuracy, valid_loss

In [16]:
def main_fun(model, train_loader, valid_loader, augment_ds_incr = False):
    if CFG.wandb:
        os.environ["WANDB_API_KEY"] = CFG.api
        wandb.init(project=CFG.project, entity=CFG.entity, reinit=True, config=class2dict(CFG))
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    kwargs = {'num_workers': CFG.num_workers, 'pin_memory': True} if \
    use_cuda else {}
    
    model = model.to(device)
    
    for epoch in range(1, CFG.num_epochs + 1):
        if augment_ds_incr:
            train_loader, valid_loader = init_ds(df_train_cut, 
                                                 path_ds=PATH_DS, 
                                                 augment=True) ## VV: for DS increase random aug generation before each epoch
        print('\nEpoch:', epoch)
        
        train(model, train_loader)
        valid(model, valid_loader)
          
    print('------------- FINISH ----------------')

Попробуем предобученные модели в режиме fine tuning:

In [17]:
MODE = 'none' #'model_selection' ## 'model_selection', 'model_freeze_test'

model_list = ['resnet',
              'alexnet',
              'vgg11',
              'vgg16',
              'vgg19',
              'squeezenet',
              'densenet']    

trained_model_list = {}


if MODE == 'model_selection':

    for model in model_list:
    
       model_1 = initialize_model(model, 2, 
                                    feature_extract=True, use_pretrained=True)[0]
       model_2 = RSNA_model(model_1, csv_use = False)
       main_fun(model_2, train_loader, valid_loader)
       trained_model_list[model] = model_2

Проверим эффект аугментации и добавления метаинформации на результат работы лучшей модели (vgg11_bn).

In [18]:
model_1 = initialize_model('vgg11', 2, 
                                    feature_extract=True, use_pretrained=True)[0]

model_vgg11_feat = RSNA_model(model_1, csv_use = True)
model_vgg11_feat_aug = RSNA_model(model_1, csv_use = True)

main_fun(model_vgg11_feat, train_loader, valid_loader) ## с метаинформацией без аугментации
main_fun(model_vgg11_feat_aug, train_loader, valid_loader, augment_ds_incr = True) ## c метаинформацией и с аугментацией


Epoch: 1


  0%|          | 0/69 [00:00<?, ?it/s]

Train set: Average loss: 0.6266, Accuracy: 68.64%


  0%|          | 0/25 [00:00<?, ?it/s]

Valid set: Average loss: 0.5936, Accuracy: 68.30%

Epoch: 2


...

Valid set: Average loss: 0.4920, Accuracy: 73.00%

Epoch: 18


  0%|          | 0/69 [00:00<?, ?it/s]

Train set: Average loss: 0.4827, Accuracy: 72.93%


  0%|          | 0/25 [00:00<?, ?it/s]

Valid set: Average loss: 0.4844, Accuracy: 72.40%
------------- FINISH ----------------


In [19]:
## nice explanation! https://discuss.pytorch.org/t/how-the-pytorch-freeze-network-in-some-layers-only-the-rest-of-the-training/7088/2
################## all layers unfreezing #####################
MODE = 'none' #'model_selection'
model_1 = initialize_model('vgg11', 2, 
                                    feature_extract=False, use_pretrained=True)[0]

model_2 = initialize_model('vgg11', 2, 
                                feature_extract=True, use_pretrained=True)[0]

summary(model =  model_1,
        input_size=(32,3,224,224),
        col_names = ['input_size','output_size','num_params','trainable'])

if MODE == 'model_selection':
    model_vgg_unfreezed = RSNA_model(model_1, csv_use = True)

################## fully-connected layers unfreezing #####################

    count = 0
    for child in model_1.children():
        if count < 2:
            for param in child.parameters():
                param.requires_grad = False
        count+=1

    model_vgg_fc_unfreezed = RSNA_model(model_1, csv_use = True)

################## last fully-connected layer unfreezing #####################

## 

## VV - to count layers
    count = 0
    for param in model_2.parameters():
        print(str(count) + '_'+ str(len(param)))
        count+=1

    count = 0
    for param in model_2.parameters():
        if count > 33:
            param.requires_grad = True
        count += 1


    model_vgg_last_fc_unfreezed = RSNA_model(model_2, csv_use = True)

    vgg_model_list = {'model_vgg_unfreezed'         : model_vgg_unfreezed,
                      'model_vgg_fc_unfreezed'      : model_vgg_fc_unfreezed,
                      'model_vgg_last_fc_unfreezed' : model_vgg_last_fc_unfreezed
        
    }
    for model in vgg_model_list.values():
        main_fun(model, train_loader, valid_loader)

## **4.2** Оценка метрик

In [20]:
from sklearn.metrics import roc_auc_score

## VV: probabilistic F-score https://www.kaggle.com/code/sohier/probabilistic-f-score
def pfbeta(labels, predictions, beta=1): ##VV: beta=1 based on 1st place notebook
    y_true_count = 0
    ctp = 0
    cfp = 0

    for idx in range(len(labels)):
        prediction = min(max(predictions[idx], 0), 1)
        if (labels[idx]):
            y_true_count += 1
            ctp += prediction
        else:
            cfp += prediction

    beta_squared = beta * beta
    c_precision = ctp / (ctp + cfp)
    c_recall = ctp / y_true_count
    if (c_precision > 0 and c_recall > 0):
        result = (1 + beta_squared) * (c_precision * c_recall) / (beta_squared * c_precision + c_recall)
        return result
    else:
        return 0

In [21]:
def metric_calc(model, loader):
    with torch.no_grad():
        model_test = model.eval() #trained_model_list['vgg11_csv']
        preds, obs = [], []
        for k, data in enumerate(loader): # valid_loader):
            image, meta, target = data
            image, meta, target = image.to(device), meta.to(device), target.to(device)
            out = model_test(image, meta)
            pred = out.argmax(dim=1, keepdim=True)
            preds.append(pred.flatten().detach().cpu().numpy())
            obs.append(target.detach().cpu().numpy())
            
        obs = [item for sublist in obs for item in sublist]
        preds = [item for sublist in preds for item in sublist]
        
        roc_auc = roc_auc_score(obs, preds)
        pf_beta = pfbeta(obs, preds)
        
        return roc_auc, pf_beta


In [24]:
best_model = model_vgg11_feat_aug

roc_tr, F_tr = metric_calc(best_model,train_loader)
roc_val, F_val = metric_calc(best_model,valid_loader)
print(f'TRAINING: aucroc: {round(roc_tr,2)}, probability F-score:{round(F_tr,2)}')
print('--------------------------------------------------------------------')
print(f'VALID: aucroc: {round(roc_val,2)}, probability F-score:{round(F_val,2)}')

TRAINING: aucroc: 0.72, probability F-score:0.62
--------------------------------------------------------------------
VALID: aucroc: 0.72, probability F-score:0.62


Теперь посчитаем различные метрики для оценки качества моделей:  rocauc и probabilistic F-score (последний является ключевым критерием в соревновании.
Сохраним оптимальную модель

In [25]:
torch.save(best_model,'/kaggle/working/best_model.pt')